In [ ]:
import numpy as np


class Window:

    def __init__(self, window_min, window_max):
        self.window_min = window_min
        self.window_max = window_max

    def __call__(self, image):
        image = np.clip(image, self.window_min, self.window_max)

        return image


class MinMaxNorm:

    def __init__(self, low, high):
        self.low = low
        self.high = high

    def __call__(self, image):
        image = (image - self.low) / (self.high - self.low)
        image = image * 2 - 1

        return image

In [ ]:
import os
from itertools import product

import nibabel as nib
import numpy as np
import torch
from skimage.measure import regionprops
from torch.utils.data import DataLoader, Dataset


class FracNetTrainDataset(Dataset):

    def __init__(self, image_dir, label_dir=None, crop_size=64,
            transforms=None, num_samples=4, train=True):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.public_id_list = sorted([x.split("-")[0]
            for x in os.listdir(image_dir)])
        self.crop_size = crop_size
        self.transforms = transforms
        self.num_samples = num_samples
        self.train = train

    def __len__(self):
        return len(self.public_id_list)

    @staticmethod
    def _get_pos_centroids(label_arr):
        centroids = [tuple([round(x) for x in prop.centroid])
            for prop in regionprops(label_arr)]

        return centroids

    @staticmethod
    def _get_symmetric_neg_centroids(pos_centroids, x_size):
        sym_neg_centroids = [(x_size - x, y, z) for x, y, z in pos_centroids]

        return sym_neg_centroids

    @staticmethod
    def _get_spine_neg_centroids(shape, crop_size, num_samples):
        x_min, x_max = shape[0] // 2 - 40, shape[0] // 2 + 40
        y_min, y_max = 300, 400
        z_min, z_max = crop_size // 2, shape[2] - crop_size // 2
        spine_neg_centroids = [(
            np.random.randint(x_min, x_max),
            np.random.randint(y_min, y_max),
            np.random.randint(z_min, z_max)
        ) for _ in range(num_samples)]

        return spine_neg_centroids

    def _get_neg_centroids(self, pos_centroids, image_shape):
        num_pos = len(pos_centroids)
        sym_neg_centroids = self._get_symmetric_neg_centroids(
            pos_centroids, image_shape[0])

        if num_pos < self.num_samples // 2:
            spine_neg_centroids = self._get_spine_neg_centroids(image_shape,
                self.crop_size, self.num_samples - 2 * num_pos)
        else:
            spine_neg_centroids = self._get_spine_neg_centroids(image_shape,
                self.crop_size, num_pos)

        return sym_neg_centroids + spine_neg_centroids

    def _get_roi_centroids(self, label_arr):
        if self.train:
            # generate positive samples' centroids
            pos_centroids = self._get_pos_centroids(label_arr)

            # generate negative samples' centroids
            neg_centroids = self._get_neg_centroids(pos_centroids,
                label_arr.shape)

            # sample positives and negatives when necessary
            num_pos = len(pos_centroids)
            num_neg = len(neg_centroids)
            if num_pos >= self.num_samples:
                num_pos = self.num_samples // 2
                num_neg = self.num_samples // 2
            elif num_pos >= self.num_samples // 2:
                num_neg = self.num_samples - num_pos

            if num_pos < len(pos_centroids):
                pos_centroids = [pos_centroids[i] for i in np.random.choice(
                    range(0, len(pos_centroids)), size=num_pos, replace=False)]
            if num_neg < len(neg_centroids):
                neg_centroids = [neg_centroids[i] for i in np.random.choice(
                    range(0, len(neg_centroids)), size=num_neg, replace=False)]

            roi_centroids = pos_centroids + neg_centroids
        else:
            roi_centroids = [list(range(0, x, y // 2))[1:-1] + [x - y // 2]
                for x, y in zip(label_arr.shape, self.crop_size)]
            roi_centroids = list(product(*roi_centroids))

        roi_centroids = [tuple([int(x) for x in centroid])
            for centroid in roi_centroids]

        return roi_centroids

    def _crop_roi(self, arr, centroid):
        roi = np.ones(tuple([self.crop_size] * 3)) * (-1024)

        src_beg = [max(0, centroid[i] - self.crop_size // 2)
            for i in range(len(centroid))]
        src_end = [min(arr.shape[i], centroid[i] + self.crop_size // 2)
            for i in range(len(centroid))]
        dst_beg = [max(0, self.crop_size // 2 - centroid[i])
            for i in range(len(centroid))]
        dst_end = [min(arr.shape[i] - (centroid[i] - self.crop_size // 2),
            self.crop_size) for i in range(len(centroid))]
        roi[
            dst_beg[0]:dst_end[0],
            dst_beg[1]:dst_end[1],
            dst_beg[2]:dst_end[2],
        ] = arr[
            src_beg[0]:src_end[0],
            src_beg[1]:src_end[1],
            src_beg[2]:src_end[2],
        ]

        return roi

    def _apply_transforms(self, image):
        for t in self.transforms:
            image = t(image)

        return image

    def __getitem__(self, idx):
        # read image and label
        public_id = self.public_id_list[idx]
        image_path = os.path.join(self.image_dir, f"{public_id}-image.nii")
        label_path = os.path.join(self.label_dir, f"{public_id}-label.nii")
        image = nib.load(image_path)
        label = nib.load(label_path)
        image_arr = image.get_fdata().astype(float)
        label_arr = label.get_fdata().astype(np.uint8)

        # calculate rois' centroids
        roi_centroids = self._get_roi_centroids(label_arr)

        # crop rois
        image_rois = [self._crop_roi(image_arr, centroid)
            for centroid in roi_centroids]
        label_rois = [self._crop_roi(label_arr, centroid)
            for centroid in roi_centroids]

        if self.transforms is not None:
            image_rois = [self._apply_transforms(image_roi)
                for image_roi in image_rois]

        image_rois = torch.tensor(np.stack(image_rois)[:, np.newaxis],
            dtype=torch.float)
        label_rois = (np.stack(label_rois) > 0).astype(float)
        label_rois = torch.tensor(label_rois[:, np.newaxis],
            dtype=torch.float)
        label_rois =label_rois.squeeze()
        return image_rois, label_rois

    @staticmethod
    def collate_fn(samples):
        image_rois = torch.cat([x[0] for x in samples])
        label_rois = torch.cat([x[1] for x in samples])

        return image_rois, label_rois

    @staticmethod
    def get_dataloader(dataset, batch_size, shuffle=False, num_workers=0):
        return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle,
            num_workers=num_workers, collate_fn=FracNetTrainDataset.collate_fn)


class FracNetInferenceDataset(Dataset):

    def __init__(self, image_path, crop_size=64, transforms=None):
        image = nib.load(image_path)
        self.image_affine = image.affine
        self.image = image.get_fdata().astype(np.int16)
        self.crop_size = crop_size
        self.transforms = transforms
        self.centers = self._get_centers()

    def _get_centers(self):
        dim_coords = [list(range(0, dim, self.crop_size // 2))[1:-1]\
            + [dim - self.crop_size // 2] for dim in self.image.shape]
        centers = list(product(*dim_coords))

        return centers

    def __len__(self):
        return len(self.centers)

    def _crop_patch(self, idx):
        center_x, center_y, center_z = self.centers[idx]
        patch = self.image[
            center_x - self.crop_size // 2:center_x + self.crop_size // 2,
            center_y - self.crop_size // 2:center_y + self.crop_size // 2,
            center_z - self.crop_size // 2:center_z + self.crop_size // 2
        ]

        return patch

    def _apply_transforms(self, image):
        for t in self.transforms:
            image = t(image)

        return image

    def __getitem__(self, idx):
        image = self._crop_patch(idx)
        center = self.centers[idx]

        if self.transforms is not None:
            image = self._apply_transforms(image)

        image = torch.tensor(image[np.newaxis], dtype=torch.float)

        return image, center

    @staticmethod
    def _collate_fn(samples):
        images = torch.stack([x[0] for x in samples])
        centers = [x[1] for x in samples]

        return images, centers

    @staticmethod
    def get_dataloader(dataset, batch_size, num_workers=0):
        return DataLoader(dataset, batch_size, num_workers=num_workers,
            collate_fn=FracNetInferenceDataset._collate_fn)

In [ ]:
import torch
import torch.nn as nn


class UNet(nn.Module):
    def __init__(self, in_channels, num_classes, first_out_channels=16):
        super().__init__()
        self.first = ConvBlock(in_channels, first_out_channels)
        in_channels = first_out_channels
        self.down1 = Down(in_channels, 2 * in_channels)
        self.down2 = Down(2 * in_channels, 4 * in_channels)
        self.down3 = Down(4 * in_channels, 8 * in_channels)
        self.up1   = Up(8 * in_channels, 4 * in_channels)
        self.up2   = Up(4 * in_channels, 2 * in_channels)
        self.up3   = Up(2 * in_channels, in_channels)
        self.final = nn.Conv3d(in_channels, num_classes, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
         # If there's an extra dimension, squeeze it
        if x.dim() == 6:
            x = x.squeeze(2)  # Remove the extra dimension

            print("Input shape after squeeze:", x.shape)
        
        x1 = self.first(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x  = self.up1(x4, x3)
        x  = self.up2(x, x2)
        x  = self.up3(x, x1)
        x  = self.final(x)
        return x


class ConvBlock(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super().__init__(
            nn.Conv3d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )


class Down(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super().__init__(
            nn.MaxPool3d(2),
            ConvBlock(in_channels, out_channels)
        )


class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, out_channels)
        self.conv2 = nn.Sequential(
            nn.ConvTranspose3d(in_channels, out_channels, 2, stride=2, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x, y):
        
        x = self.conv2(x)
        x = self.conv1(torch.cat([y, x], dim=1))
        return x

In [ ]:

from functools import reduce

import torch
import torch.nn as nn
import torch.nn.functional as F


__all__ = ['MixLoss', 'DiceLoss', 'GHMCLoss', 'FocalLoss']


class MixLoss(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.args = args

    def forward(self, x, y):
        lf, lfw = [], []
        for i, v in enumerate(self.args):
            if i % 2 == 0:
                lf.append(v)
            else:
                lfw.append(v)
        mx = sum([w * l(x, y) for l, w in zip(lf, lfw)])
        return mx


class DiceLoss(nn.Module):
    def __init__(self, image=False):
        super().__init__()
        self.image = image

    def forward(self, x, y):
        x = x.sigmoid()
        i, u = [t.flatten(1).sum(1) if self.image else t.sum() for t in [x * y, x + y]]

        dc = (2 * i + 1) / (u + 1)
        dc = 1 - dc.mean()
        return dc


class GHMCLoss(nn.Module):
    def __init__(self, mmt=0, bins=10):
        super().__init__()
        self.mmt = mmt
        self.bins = bins
        self.edges = [x / bins for x in range(bins + 1)]
        self.edges[-1] += 1e-6

        if mmt > 0:
            self.acc_sum = [0] * bins

    def forward(self, x, y):
        w = torch.zeros_like(x)
        g = torch.abs(x.detach().sigmoid() - y)

        n = 0
        t = reduce(lambda x, y: x * y, w.shape)
        for i in range(self.bins):
            ix = (g >= self.edges[i]) & (g < self.edges[i + 1]); nb = ix.sum()
            if nb > 0:
                if self.mmt > 0:
                    self.acc_sum[i] = self.mmt * self.acc_sum[i] + (1 - self.mmt) * nb
                    w[ix] = t / self.acc_sum[i]
                else:
                    w[ix] = t / nb
                n += 1
        if n > 0:
            w = w / n

        gc = F.binary_cross_entropy_with_logits(x, y, w, reduction='sum') / t
        return gc


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, x, y):
        ce = F.binary_cross_entropy_with_logits(x, y)
        fc = self.alpha * (1 - torch.exp(-ce)) ** self.gamma * ce
        return fc

In [ ]:

def dice(x, y, image=False):
    x = x.sigmoid()
    i, u = [t.flatten(1).sum(1) if image else t.sum() for t in [x * y, x + y]]
    dc = ((2 * i + 1) / (u + 1)).mean()
    return dc


def recall(x, y, thresh=0.1):
    x = x.sigmoid()
    tp = (((x * y) > thresh).flatten(1).sum(1) > 0).sum()
    rc = tp / (((y > 0).flatten(1).sum(1) > 0).sum() + 1e-8)
    return rc


def accuracy(x, y, thresh=0.5):
    x = x.sigmoid()
    ac = ((x > thresh) == (y > 0)).float().mean()
    return ac


def precision(x, y, thresh=0.1):
    x = x.sigmoid()
    tp = (((x * y) > thresh).flatten(1).sum(1) > 0).sum()
    pc = tp / (((x > thresh).flatten(1).sum(1) > 0).sum() + 1e-8)
    return pc


def fbeta_score(x, y, beta=1, **kwargs):
    rc = recall(x, y, **kwargs)
    pc = precision(x, y, **kwargs)
    fs = (1 + beta ** 2) * pc * rc / (beta ** 2 * pc + rc + 1e-8)
    return fs

In [ ]:
from functools import partial
from fastai.callback.tensorboard import TensorBoardCallback
from fastai.vision.all import *
import torch
import torch.nn.functional as F
from torchmetrics import Recall, Precision, Dice, FBetaScore
from torch import nn

class TransformCallback(Callback):
    def before_batch(self):
        if isinstance(self.learn.xb, tuple):
            x = self.learn.xb[0]
        else:
            x = self.learn.xb
            
        if isinstance(self.learn.yb, tuple):
            y = self.learn.yb[0]
        else:
            y = self.learn.yb
        
        # Handle input shape: [B, 4, 1, 64, 64, 64]
        x = x.squeeze(2)  # Remove singleton dimension
        
        # Handle target shape: [B, 4, 64, 64, 64]
        # Assuming this is a one-hot encoded target, convert to class indices
        y = y.argmax(dim=1)
        
        self.learn.xb = (x,)
        self.learn.yb = (y,)
    
    def after_pred(self):
        pred = self.learn.pred[0] if isinstance(self.learn.pred, tuple) else self.learn.pred
        target = self.learn.yb[0] if isinstance(self.learn.yb, tuple) else self.learn.yb
        
        self.learn.pred = pred
        self.learn.yb = (target,)

def dice_score(input, target, epsilon=1e-6):
    # input shape: [B, C, H, W, D] - model output (5 classes)
    # target shape: [B, H, W, D] - class indices
    probs = F.softmax(input, dim=1)
    pred_classes = probs.argmax(dim=1)
    
    # Compute dice score
    intersection = (pred_classes == target).float().sum()
    dice = (2. * intersection + epsilon) / (pred_classes.numel() + target.numel() + epsilon)
    
    return dice

def multi_class_recall(input, target, num_classes=5):
    # input shape: [B, C, H, W, D] - model output (5 classes)
    # target shape: [B, H, W, D] - class indices
    probs = F.softmax(input, dim=1)
    pred_classes = probs.argmax(dim=1)
    
    # Initialize recall for each class
    recalls = []
    for cls in range(num_classes):
        # Create binary masks for the current class
        pred_mask = (pred_classes == cls).float()
        target_mask = (target == cls).float()
        
        # Compute recall for the current class
        true_positives = (pred_mask * target_mask).sum()
        actual_positives = target_mask.sum()
        
        # Avoid division by zero
        recall = true_positives / (actual_positives + 1e-6)
        recalls.append(recall)
    
    # Return mean recall across classes
    return torch.tensor(recalls).mean()

def multi_class_precision(input, target, num_classes=5):
    # input shape: [B, C, H, W, D] - model output (5 classes)
    # target shape: [B, H, W, D] - class indices
    probs = F.softmax(input, dim=1)
    pred_classes = probs.argmax(dim=1)
    
    # Initialize precision for each class
    precisions = []
    for cls in range(num_classes):
        # Create binary masks for the current class
        pred_mask = (pred_classes == cls).float()
        target_mask = (target == cls).float()
        
        # Compute precision for the current class
        true_positives = (pred_mask * target_mask).sum()
        predicted_positives = pred_mask.sum()
        
        # Avoid division by zero
        precision = true_positives / (predicted_positives + 1e-6)
        precisions.append(precision)
    
    # Return mean precision across classes
    return torch.tensor(precisions).mean()

def multi_class_fbeta_score(input, target, beta=1, num_classes=5):
    # input shape: [B, C, H, W, D] - model output (5 classes)
    # target shape: [B, H, W, D] - class indices
    probs = F.softmax(input, dim=1)
    pred_classes = probs.argmax(dim=1)
    
    # Initialize F-Beta scores for each class
    fbeta_scores = []
    for cls in range(num_classes):
        # Create binary masks for the current class
        pred_mask = (pred_classes == cls).float()
        target_mask = (target == cls).float()
        
        # Compute true positives, false positives, and false negatives
        true_positives = (pred_mask * target_mask).sum()
        false_positives = ((pred_mask > 0) * (target_mask == 0)).sum()
        false_negatives = ((pred_mask == 0) * (target_mask > 0)).sum()
        
        # Compute precision and recall
        precision = true_positives / (true_positives + false_positives + 1e-6)
        recall = true_positives / (true_positives + false_negatives + 1e-6)
        
        # Compute F-Beta score
        # F-Beta = (1 + beta^2) * (precision * recall) / ((beta^2 * precision) + recall)
        beta_sq = beta ** 2
        fbeta = ((1 + beta_sq) * precision * recall) / ((beta_sq * precision) + recall + 1e-6)
        
        fbeta_scores.append(fbeta)
    
    # Return mean F-Beta score across classes
    return torch.tensor(fbeta_scores).mean()

class CombinedLoss(nn.Module):
    def __init__(self, weights=[0.5, 0.5]):
        super().__init__()
        self.weights = weights
        self.ce = nn.CrossEntropyLoss()
    
    def forward(self, input, target):
        # input: [B, C, H, W, D] - 5 classes
        # target: [B, H, W, D] - class indices
        ce_loss = self.ce(input, target)
        
        # Dice Loss calculation
        probs = F.softmax(input, dim=1)
        pred_classes = probs.argmax(dim=1)
        
        # One-hot encode target
        encoded_target = F.one_hot(target, num_classes=input.shape[1])
        encoded_target = encoded_target.permute(0, 4, 1, 2, 3).float()
        
        intersection = (probs * encoded_target).sum(dim=(2, 3, 4))
        union = probs.sum(dim=(2, 3, 4)) + encoded_target.sum(dim=(2, 3, 4))
        
        dice_loss = 1 - ((2. * intersection + 1e-6) / (union + 1e-6)).mean()
        
        return self.weights[0] * ce_loss + self.weights[1] * dice_loss

def main():
    train_image_dir = "/kaggle/input/ribfrac2/train_image"
    train_label_dir = "/kaggle/input/ribfrac2/train_label"
    val_image_dir = "/kaggle/input/ribfrac2/val_image"
    val_label_dir = "/kaggle/input/ribfrac2/val_label"
    
    batch_size = 2  # Match your actual batch size
    num_workers = 4
    
    # Model with input channels matching your data
    model = UNet(in_channels=4, num_classes=5, first_out_channels=64)
    model = nn.DataParallel(model.cuda())
    
    # Transforms
    transforms = [
        Window(-200, 1000),
        MinMaxNorm(-200, 1000)
    ]
    
    # Datasets and DataLoaders
    ds_train = FracNetTrainDataset(
        train_image_dir, 
        train_label_dir,
        transforms=transforms
    )
    
    ds_val = FracNetTrainDataset(
        val_image_dir, 
        val_label_dir,
        transforms=transforms
    )
    
    dl_train = FracNetTrainDataset.get_dataloader(
        ds_train, 
        batch_size, 
        shuffle=True,
        num_workers=num_workers
    )
    
    dl_val = FracNetTrainDataset.get_dataloader(
        ds_val, 
        batch_size, 
        shuffle=False,
        num_workers=num_workers
    )
    
    # Create DataLoaders
    dls = DataLoaders(dl_train, dl_val, device=default_device())
    
    # Loss function
    loss_func = CombinedLoss(weights=[0.5, 0.5])
    
    # Create learner with additional metrics
    learn = Learner(
        dls,
        model,
        loss_func=loss_func,
        metrics=[
            dice_score,               # Dice score metric 
            multi_class_recall,        # Recall metric
            multi_class_precision,     # Precision metric
            partial(multi_class_fbeta_score, beta=1),  # F1 Score 
            partial(multi_class_fbeta_score, beta=2)   # F2 Score
        ],
        opt_func=Adam
    )
    
    # Add callbacks
    learn.add_cb(TransformCallback())
    learn.add_cb(SaveModelCallback(monitor='dice_score', comp=np.greater))
    
    # Train model
    try:
        learn.fit_one_cycle(
            40,
            1e-3,
            cbs=[GradientClip(3.0)]
        )
    except Exception as e:
        print(f"Training error: {str(e)}")
        batch = next(iter(dls.train))
        x, y = batch
        print(f"Input shape: {x.shape}, Target shape: {y.shape}")
        raise e

    # Save final model
    torch.save(model.module.state_dict(), "final_model.pth")

if __name__ == "__main__":
    main()

In [ ]:
import random

def generate_dataset_summary():
    print("Variables".ljust(30), "Internal training set".ljust(25), 
          "Internal verification set".ljust(25), "External verification set")
    print("="*100)

    # Randomly generating data for demonstration
    num_patients = [random.randint(9000, 12000), random.randint(1500, 2000), random.randint(1500, 2000)]
    num_positive = [random.randint(7000, 8000), random.randint(300, 500), random.randint(800, 1000)]
    num_negative = [p - pos for p, pos in zip(num_patients, num_positive)]
    
    num_bone_kernels = [random.randint(2000, 3000), random.randint(800, 900), random.randint(500, 600)]
    bone_percent = [f"{(k / p) * 100:.1f}%" for k, p in zip(num_bone_kernels, num_patients)]
    num_standard_kernels = [p - k for p, k in zip(num_patients, num_bone_kernels)]
    std_percent = [f"{(k / p) * 100:.1f}%" for k, p in zip(num_standard_kernels, num_patients)]

    num_slices = [[random.randint(500, 800), random.randint(3000, 3500), random.randint(7000, 8000)] for _ in range(3)]
    mean_age = [round(random.uniform(48, 54), 1) for _ in range(3)]
    sex_ratios = [[random.randint(60, 65), 100 - random.randint(60, 65)] for _ in range(3)]
    
    num_fractures = [random.randint(35000, 40000), random.randint(2000, 2500), random.randint(4000, 5000)]
    displaced = [random.randint(8000, 9000), random.randint(400, 500), random.randint(800, 1000)]
    non_displaced = [random.randint(10000, 12000), random.randint(500, 600), random.randint(1300, 1500)]
    buckle = [random.randint(6000, 7000), random.randint(300, 400), random.randint(1100, 1300)]
    old_fractures = [random.randint(10000, 12000), random.randint(700, 900), random.randint(800, 1000)]

    # Print data row by row
    def format_row(label, data, suffix=""):
        return label.ljust(30) + "".join([str(x).ljust(25) + suffix for x in data])
    
    print(format_row("No. of patients", num_patients))
    print(format_row("No. of positive patients", num_positive))
    print(format_row("No. of negative patients", num_negative))
    print(format_row("No. of bone kernels", [f"{k} ({p})" for k, p in zip(num_bone_kernels, bone_percent)]))
    print(format_row("No. of standard kernels", [f"{k} ({p})" for k, p in zip(num_standard_kernels, std_percent)]))
    print(format_row("No. of slices (> 2 mm)", [n[0] for n in num_slices]))
    print(format_row("No. of slices (<= 2 mm, > 1 mm)", [n[1] for n in num_slices]))
    print(format_row("No. of slices (<= 1 mm)", [n[2] for n in num_slices]))
    print(format_row("Mean age (range)", mean_age, " years"))
    print(format_row("Sex (male : female)", [f"{r[0]}% : {r[1]}%" for r in sex_ratios]))
    print(format_row("No. of fractures", num_fractures))
    print(format_row("No. of displaced fractures", displaced))
    print(format_row("No. of non-displaced fractures", non_displaced))
    print(format_row("No. of buckle fractures", buckle))
    print(format_row("No. of old fractures", old_fractures))

# Run the script
generate_dataset_summary()